In [13]:
import os
import torch
# import pandas as pd
# from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import json
import io
from PIL import Image

In [10]:

class RndDataloader(Dataset):

    def __init__(self, root: str=None, train=True, transform=None, target_transform = None,split : float = 0.5)-> None:
        super().__init__()
        self.train = train 
        self.split = split
        self.height=96
        self.width=96
        self.transform = transform
        self.classes = {
            'AllenKey': 0,
            'Axis': 1,
            'Bearing2': 2,
            'Drill': 3,
            'F20_20_B': 4,
            'F20_20_G': 5,
            'Housing': 6,
            'M20': 7,
            'M20_100': 8,
            'M30': 9,
            'Motor2': 10,
            'S40_40_B': 11,
            'S40_40_G': 12,
            'Screwdriver': 13,
            'Spacer': 14,
            'Wrench': 15,
            'container_blue': 16,
            'container_red': 17,
        }
        self.folder_path = '/home/gchenc2s/src/rnd_gokulkrishna/dataset/sample/sample_set_annotated/'
        self.json_files, self.json_data  = self.load_data()
        # self.test()

    def __len__(self):
        return len(self.folder_path)
    
    def __getitem__(self, idx_label_no):
        idx, label_no = idx_label_no
        points = []
        img_path = self.json_files[idx].split('.')[0] + '.jpg'
        path = os.path.basename(img_path)
        img = Image.open(img_path)
        res_x, res_y = img.size
        img = img.resize((self.height, self.width), Image.BILINEAR)
        # print(path, img.size)
        convert = transforms.ToTensor()
        img = convert(img)
        ###### take the label_no and fetch data from json file or data ######
        ###### use this json data and then from the dictionary petch all the classes available ######
        ###### use the data from dictionary and label_no to get the points from json_data ######
        
        for entry in self.json_data:
            if entry['image'] == path:
                print(entry['data'])
                for data_item in entry['data']:
                    label = data_item['label']
                    # if label == self.classes[label_no]:
                    if label == label_no:
                        points.append(data_item['points'])
                        break
        points = np.array(points[0])
        points = [[point[0]/res_x, point[1]/res_y] for point in points]
        

        return img, points
        
    
    def load_data(self):
        folder_path = self.folder_path
        data_load = []
        json_files = []
        for file in os.listdir(folder_path):
            if file.endswith(".jpg"):
                # image_path = os.path.join(folder_path, file)
                json_path = os.path.join(folder_path, file.split('.')[0] + '.json')
                
                with open(json_path) as f:
                    json_data = json.load(f)
                    json_files.append(json_path)
                    data = []
                    for shape in json_data['shapes']:
                        label = shape['label']
                        points = shape['points']
                        data.append({'label': label, 'points': points})
                    data_load.append({'image': file, 'data': data})
        print(data_load)
        # print(json_files)
        return json_files, data_load
    
    # def test(self):
    #     print(self.folder_path)



In [11]:
folder_path = '/home/gchenc2s/src/rnd_gokulkrishna/dataset/sample/sample_set_annotated/'
# path needs to be changed in __init__ of RndDataloader
rndloader = RndDataloader(train=True)

In [ ]:
sample_indices = 6
sample_label = 'S40_40_B'
# image,point = rndloader[sample_indices][ sample_label]
image , point = rndloader[sample_indices, sample_label]
# Image.fromarray(image).show()
print(point)

# Cuurent output
'''[{'label': 'S40_40_B', 'points': [[564.0, 403.0], [644.0, 504.0]]}, {'label': 'S40_40_B', 'points': [[925.0, 240.0], [1015.0, 333.0]]}, {'label': 'S40_40_G', 'points': [[404.0, 417.0], [452.0, 525.0]]}, {'label': 'S40_40_G', 'points': [[975.0, 370.0], [1051.0, 477.0]]}, {'label': 'F20_20_G', 'points': [[858.0, 452.0], [902.0, 549.0]]}]
[[0.440625, 0.5597222222222222], [0.503125, 0.7]]'''